In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import sys
sys.path.append('../evaluator')
from evaluator import load_data, prepare_data, load_and_tokenize, \
                    predict_toxicity, calculate_similarity, calculate_bleu, \
                    calculate_fluency

/home/jovyan/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


#### Backtranslation + Paraphrased

In [2]:
bart_concat = pd.read_csv('../data/parallel/output/bart_concat_eval.txt', sep='\t')
bart_concat.head()

,source,gen,preds,sem_similarity,bleu,fluency
0,you are a fucking idiot .,You're kidding.,0,0.329983,0.000000,0
1,as stupid and arrogant as his boss,"As a boss,",0,0.638569,0.000000,1
2,a stupid society does stupid things and votes ...,A silly society does things and votes for craz...,1,0.815421,0.298125,0
3,a president who is an idiot .,A president who's a little crazy.,1,0.749055,0.000000,0
4,colbert is a stupid moron and is a terrorist .,Colbert is a crazy man and is a terrorist.,1,0.898991,0.525382,0


In [55]:
bart_concat[bart_concat['source'].str.contains('oh i am so scared , asshole .')]

,source,gen,preds,sem_similarity,bleu,fluency
5921,"oh i am so scared , asshole .","Oh, I'm so scared, man.",0,0.707968,0.0,0


In [3]:
sta_gt = np.zeros(len(bart_concat), dtype=int)
sta_fl = np.ones(len(bart_concat), dtype=int)
print(f'Accuracy score: {accuracy_score(sta_gt, bart_concat["preds"].to_numpy()):.3f}')
print(f'Semantic similarity: {bart_concat["sem_similarity"].mean():.3f}')
print(f'BLEU: {bart_concat["bleu"].mean():.3f}')
print(f'Fluency: {accuracy_score(sta_fl, bart_concat["fluency"].to_numpy()):.3f}')

Accuracy score: 0.531
Semantic similarity: 0.825
BLEU: 0.459
Fluency: 0.164


#### Backtranslation

In [40]:
bart_bt = pd.read_csv('../data/parallel/output/bart_bt_eval.txt', sep='\t')
bart_bt.head()

,source,gen,preds,sem_similarity,bleu,fluency
0,you are a fucking idiot .,You're an idiot.,1,0.840722,0.000000,0
1,as stupid and arrogant as his boss,As arrogant as his boss.,1,0.926194,0.454802,0
2,a stupid society does stupid things and votes ...,A silly society does silly things and votes fo...,1,0.877173,0.585906,0
3,a president who is an idiot .,A president who's an idiot.,1,0.986281,0.000000,0
4,colbert is a stupid moron and is a terrorist .,Colbert is a crazy man and he's a terrorist.,1,0.886555,0.000000,0


In [57]:
bart_bt[bart_bt['source'].str.contains('oh i am so scared , asshole .')]

,source,gen,preds,sem_similarity,bleu,fluency
5921,"oh i am so scared , asshole .","Oh, I'm so scared, asshole.",1,0.948957,0.541082,0


In [5]:
sta_gt = np.zeros(len(bart_concat), dtype=int)
sta_fl = np.ones(len(bart_concat), dtype=int)
print(f'Accuracy score: {accuracy_score(sta_gt, bart_bt["preds"].to_numpy()):.3f}')
print(f'Semantic similarity: {bart_bt["sem_similarity"].mean():.3f}')
print(f'BLEU: {bart_bt["bleu"].mean():.3f}')
print(f'Fluency: {accuracy_score(sta_fl, bart_bt["fluency"].to_numpy()):.3f}')

Accuracy score: 0.400
Semantic similarity: 0.844
BLEU: 0.494
Fluency: 0.159


#### Paraphrased

In [13]:
bart_paraphrased = pd.read_csv('../data/parallel/output/bart_gpt20k_eval.txt', sep='\t')
bart_paraphrased.head()

,source,gen,preds,sem_similarity,bleu,fluency
0,you are a fucking idiot .,You're a fool.,1,0.618262,0.000000,0
1,as stupid and arrogant as his boss,As stupid and arrogant as his boss.,1,0.988236,0.840896,0
2,a stupid society does stupid things and votes ...,A stupid society does things and votes for pol...,1,0.970699,0.472694,0
3,a president who is an idiot .,A president who is a joke.,1,0.772565,0.434721,0
4,colbert is a stupid moron and is a terrorist .,Colbert is a mistake and is a terrorist.,1,0.890646,0.534174,1


In [58]:
bart_paraphrased[bart_paraphrased['source'].str.contains('oh i am so scared , asshole .')]

,source,gen,preds,sem_similarity,bleu,fluency
5921,"oh i am so scared , asshole .","oh I'm so scared, I'm afraid.",0,0.728648,0.0,0


In [7]:
sta_gt = np.zeros(len(bart_concat), dtype=int)
sta_fl = np.ones(len(bart_concat), dtype=int)
print(f'Accuracy score: {accuracy_score(sta_gt, bart_paraphrased["preds"].to_numpy()):.3f}')
print(f'Semantic similarity: {bart_paraphrased["sem_similarity"].mean():.3f}')
print(f'BLEU: {bart_paraphrased["bleu"].mean():.3f}')
print(f'Fluency: {accuracy_score(sta_fl, bart_paraphrased["fluency"].to_numpy()):.3f}')

Accuracy score: 0.421
Semantic similarity: 0.862
BLEU: 0.570
Fluency: 0.182


#### Unsupervised CycleGAN (Adam)

In [59]:
cycgan_adam = pd.read_csv('../data/seq2seq/output/cyclegan_adam_eval.txt', sep='\t')
cycgan_adam.head()

,source,gen,preds,sem_similarity,bleu,fluency
0,stop censoring me you douche <unk> .,stop censoring me you douche <unk> .,1,1.0,1.0,0
1,i think wikipedia is a crap .,i think wikipedia is a crap .,1,1.0,1.0,0
2,"so , haha u losers lol !","so , haha u losers lol !",1,1.0,1.0,0
3,wake up your ignorance is dangerous .,wake up your ignorance is dangerous .,1,1.0,1.0,0
4,trump just employs racists and bigots .,trump just employs racists and bigots .,1,1.0,1.0,0


In [60]:
cycgan_adam[cycgan_adam['source'].str.contains('oh i am so scared , asshole .')]

,source,gen,preds,sem_similarity,bleu,fluency
4478,"oh i am so scared , asshole .","oh i am so scared , annie .",0,0.655489,0.707107,0


In [9]:
sta_gt = np.zeros(len(cycgan_adam), dtype=int)
sta_fl = np.ones(len(cycgan_adam), dtype=int)
print(f'Accuracy score: {accuracy_score(sta_gt, cycgan_adam["preds"].to_numpy()):.3f}')
print(f'Semantic similarity: {cycgan_adam["sem_similarity"].mean():.3f}')
print(f'BLEU: {cycgan_adam["bleu"].mean():.3f}')
print(f'Fluency: {accuracy_score(sta_fl, cycgan_adam["fluency"].to_numpy()):.3f}')

Accuracy score: 0.107
Semantic similarity: 0.929
BLEU: 0.924
Fluency: 0.222


#### Unsupervised CycleGAN (SGD)

In [61]:
cycgan_sgd = pd.read_csv('../data/seq2seq/output/cyclegan_sgd_eval.txt', sep='\t')
cycgan_sgd.head()

,source,gen,preds,sem_similarity,bleu,fluency
0,stop censoring me you douche <unk> .,stop censoring me you renounced <unk> .,0,0.775062,0.596949,0
1,i think wikipedia is a crap .,i think wikipedia is a crap .,1,1.000000,1.000000,0
2,"so , haha u losers lol !","so , haha u losers lol !",1,1.000000,1.000000,0
3,wake up your ignorance is dangerous .,wake up your ignorance is dangerous .,1,1.000000,1.000000,0
4,trump just employs racists and bigots .,trump just employs racists and bigots .,1,1.000000,1.000000,0


In [62]:
cycgan_sgd[cycgan_sgd['source'].str.contains('oh i am so scared , asshole .')]

,source,gen,preds,sem_similarity,bleu,fluency
4478,"oh i am so scared , asshole .","oh i am so scared , rachael .",0,0.573304,0.707107,0


In [34]:
cycgan_sgd.iloc[8204].source, cycgan_sgd.iloc[8204].gen

('if you guys keep reverting the language , i swear i will spread the word about this shit .',
 'if you guys keep reverting the language , i swear i will spread the word about this country .')

In [46]:
cycgan_sgd[(cycgan_sgd['preds'] == 0) & \
           (~cycgan_sgd['source'].str.contains('<unk>'))]\
.sort_values('bleu', ascending=False)[300:350]

,source,gen,preds,sem_similarity,bleu,fluency
6559,this website an embarrassment run by far left ...,this website an embarrassment run by far left ...,0,0.699436,0.738998,1
5008,this might have been bud blacks stupidest move...,this might have been bud blacks advances move ...,0,0.823261,0.734889,1
5236,"you are one slow and persistent motherfucker ,...","you are one slow and persistent thefts , can y...",0,0.624772,0.734889,1
5140,"wow , you re a funny motherfucker , you know t...","wow , you re a funny wayne , you know that ?",0,0.581864,0.734889,0
5155,if you have female genitalia then use the girl...,if you have female taxs then use the girls lad...,0,0.757813,0.734889,0
1191,if we ever find u. we ll torch u alive !,if we ever find u. we ll avoiding u alive !,0,0.680262,0.734889,0
5228,"and stop putting ur shitty opinions on there ,...",and stop putting ur strengths opinions on ther...,0,0.751894,0.734889,0
5417,obama is a nigger and he should nt be the pres .,obama is a dyed and he should nt be the pres .,0,0.699932,0.734889,0
3234,sort of like the cretins that signed in on the...,sort of like the dubia that signed in on the d...,0,0.696344,0.734889,1
1403,"and still , the halfwit continues to cozy up t...","and still , the onethird continues to cozy up ...",0,0.711786,0.734889,0


In [11]:
sta_gt = np.zeros(len(cycgan_adam), dtype=int)
sta_fl = np.ones(len(cycgan_adam), dtype=int)
print(f'Accuracy score: {accuracy_score(sta_gt, cycgan_sgd["preds"].to_numpy()):.3f}')
print(f'Semantic similarity: {cycgan_sgd["sem_similarity"].mean():.3f}')
print(f'BLEU: {cycgan_sgd["bleu"].mean():.3f}')
print(f'Fluency: {accuracy_score(sta_fl, cycgan_sgd["fluency"].to_numpy()):.3f}')

Accuracy score: 0.109
Semantic similarity: 0.929
BLEU: 0.925
Fluency: 0.223
